In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from os import listdir 
import numpy as np 
from PIL import Image 
import tensorflow as tf 
import keras 
from keras.applications.vgg16 import VGG16 
from sklearn.model_selection import train_test_split 
from keras.models import load_model 

Using TensorFlow backend.


In [ ]:
cats_path = '/content/drive/My Drive/BCI982/Cats/'
dogs_path = '/content/drive/My Drive/BCI982/Dogs/'
cats_img, dogs_img = [], []

In [ ]:

for i, each in enumerate(listdir(cats_path)):
    try:
        img= Image.open(cats_path + each)
        img = img.resize((224, 224))
        img= np.array(img) / 255.
        if img.shape == (224,224,3):
            cats_img.append(img)
    except:
        print("ignored loading", each)
print(len(cats_img), 'cats done')


for i, each in enumerate(listdir(dogs_path)):
    try:
        img= Image.open(dogs_path + each)
        img = img.resize((224, 224))
        img= np.array(img) / 255.
        if img.shape == (224,224,3):
            dogs_img.append(img)
    except:
        print("ignored loading", each)
print(len(dogs_img), 'dogs done')

100 cats done
100 dogs done


In [ ]:
X = cats_img + dogs_img 
y = [0] * len(cats_img) + [1] * len(dogs_img) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True) 
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, shuffle=True) 
print(len(X_train), len(y_train), len(X_val), len(y_val), len(X_test), len(y_test))

160 160 20 20 20 20


In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)
print('Train :', X_train.shape, y_train.shape) 
print('Val :', X_val.shape, y_val.shape)
print('Test :', X_test.shape, y_test.shape)

Train : (160, 224, 224, 3) (160,)
Val : (20, 224, 224, 3) (20,)
Test : (20, 224, 224, 3) (20,)


In [ ]:
model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3)) 
for each_layer in model.layers: 
    each_layer.trainable = False
interaction_output = model.layers[-1].output 
x = keras.layers.Flatten()(interaction_output) 
x = keras.layers.Dense(1024, activation="relu")(x) 
x = keras.layers.Dense(1, activation="sigmoid")(x) 
model = keras.Model(model.input, x) 
model.summary() 

58892288/58889256 [==============================] - 1s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint("/content/drive/My Drive/BCI982/model.h5", monitor='val_acc', save_best_only=True, mode='max')
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy']) 
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), batch_size = 16, callbacks=[checkpoint])
model.save('/content/drive/My Drive/BCI982/mainModel.h5' ) 

Train on 160 samples, validate on 20 samples
Epoch 1/20
160/160 [==============================] - 2s 14ms/step - loss: 2.3484 - accuracy: 0.6938 - val_loss: 0.2604 - val_accuracy: 0.9000
Epoch 2/20
 16/160 [==>...........................] - ETA: 1s - loss: 0.0231 - accuracy: 1.0000

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


160/160 [==============================] - 2s 13ms/step - loss: 0.3660 - accuracy: 0.9312 - val_loss: 0.1597 - val_accuracy: 0.9000
Epoch 3/20
160/160 [==============================] - 2s 13ms/step - loss: 0.0177 - accuracy: 0.9937 - val_loss: 0.5271 - val_accuracy: 0.9000
Epoch 4/20
160/160 [==============================] - 2s 13ms/step - loss: 1.7086e-04 - accuracy: 1.0000 - val_loss: 0.4339 - val_accuracy: 0.9000
Epoch 5/20
160/160 [==============================] - 2s 13ms/step - loss: 1.2884e-04 - accuracy: 1.0000 - val_loss: 0.3896 - val_accuracy: 0.9000
Epoch 6/20
160/160 [==============================] - 2s 13ms/step - loss: 9.3385e-05 - accuracy: 1.0000 - val_loss: 0.3699 - val_accuracy: 0.9000
Epoch 7/20
160/160 [==============================] - 2s 13ms/step - loss: 8.1471e-05 - accuracy: 1.0000 - val_loss: 0.3555 - val_accuracy: 0.9000
Epoch 8/20
160/160 [==============================] - 2s 13ms/step - loss: 6.4629e-05 - accuracy: 1.0000 - val_loss: 0.3526 - val_accurac

In [ ]:
model = load_model('/content/drive/My Drive/BCI982/mainModel.h5')


In [ ]:
_, acc = model.evaluate(X_train, y_train)
print('Train Accuracy: %.2f' % (acc*100))
_, acc = model.evaluate(X_test, y_test)
print('Test Accuracy: %.2f' % (acc*100))

160/160 [==============================] - 10s 62ms/step
Train Accuracy: 97.50
20/20 [==============================] - 2s 114ms/step
Test Accuracy: 100.00


In [ ]:
def cats_vs_dogs_classifier(address):
    answer = None
    try:
        img= Image.open(address).convert('RGB')
        img = img.resize((224, 224))
        img= np.array(img) / 255.
        if img.shape == (224,224,3):
            prediction = model.predict(np.array([img]))[0][0]
            answer = 'Cat' if round(prediction) == 0 else 'Dog'
            answer = 'it is a ' + answer + '!'
    except:
        print("ignored loading", each)
    return answer

In [ ]:
test_img_address = 'a.jpg'
print(cats_vs_dogs_classifier(test_img_address))

it is a Dog!
